In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import scipy  
import scikits.bootstrap as bootstrap
import statsmodels.stats.api as sms
from sklearn import preprocessing
from sklearn import metrics
import pandas as pd
from ggplot import *
from scipy import stats
import statsmodels.api as sm
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
import numpy as np
from sklearn.preprocessing import Imputer
from scipy.stats import variation
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.preprocessing import scale
from sklearn import cross_validation
from sklearn.preprocessing import Imputer
from sklearn.datasets import make_classification
import seaborn as sns
from sklearn.metrics import hamming_loss,accuracy_score
from sklearn import svm
from sklearn.svm import SVC
from sklearn.learning_curve import learning_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

#read the files
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

In [ ]:
import tensorflow as tf
import util

# Create the neural network
def conv_net(x_dict, height, width, reuse):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = x_dict['images']
        x = tf.reshape(x, shape=[-1, height, width, 1])
        # Convolution Layer 1 with 16 filters and a kernel size of 4 repeated twice
        conv1_1 = tf.layers.conv2d(x, 16, 5, activation=tf.nn.relu)
        conv1_p = tf.layers.max_pooling2d(conv1_1, 2, 2)
        conv1_2 = tf.layers.conv2d(conv1_p, 16, 5, activation=tf.nn.relu)
        conv1_3 = tf.layers.batch_normalization(conv1_2, axis=-1, momentum=0.99, epsilon = 0.001,
        center=True, scale=True, training=True)
        conv1_4 = tf.layers.max_pooling2d(conv1_3, 2, 2)
        # Convolution Layer 2 with 32 filters and a kernel size of 4
        conv2_1 = tf.layers.conv2d(conv1_3, 32, 3, activation=tf.nn.relu)
        conv2_p = tf.layers.max_pooling2d(conv2_1, 2, 2)
        conv2_2 = tf.layers.conv2d(conv2_p, 32, 3, activation=tf.nn.relu)
        conv2_3 = tf.layers.batch_normalization(conv2_2, axis=-1, momentum=0.99, epsilon = 0.001,
        center=True, scale=True, training=True)
        conv2_4 = tf.layers.max_pooling2d(conv2_3, 2, 2)
        # Convolution Layer 3 with 64 filters and a kernel size of 2
        conv3_1 = tf.layers.conv2d(conv2_3, 64, 2, activation=tf.nn.relu)
        conv3_p = tf.layers.max_pooling2d(conv3_1, 2, 2)
        conv3_2 = tf.layers.conv2d(conv3_1, 64, 2, activation=tf.nn.relu)                
        conv3_3 = tf.layers.batch_normalization(conv3_2, axis=-1, momentum=0.99, epsilon = 0.001,
        center=True, scale=True, training=True)        
        conv3_4 = tf.layers.max_pooling2d(conv3_3, 2, 2)
        # Output layer, a, b distribution 
        out = tf.layers.conv2d(conv3_3, 128, 1, activation=tf.nn.softmax)
        out = tf.contrib.layers.flatten(out)
    return out
        
def model_fn(features, labels, mode):
    #Build the CNN
    logits_train = conv_net(features,64,64,reuse=False)
    #If in prediction mode end early
    if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode, logits_train)

    #Define Loss and Optimizer
    labels = tf.contrib.layers.flatten(labels)
    loss_op = tf.contrib.losses.mean_squared_error(logits_train, labels)	
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
    # Evaluate the accuracy of the model 
    acc_op = tf.metrics.accuracy(labels,logits_train)

    estim_specs = tf.estimator.EstimatorSpec(mode=mode,loss=loss_op, train_op=train_op, eval_metric_ops={'accuracy': acc_op})

    return estim_specs
#--------------------------- Train Vars --------------------------------------------------------
# Training Parameters
learning_rate = 0.001
num_steps = 2000
batch_size = 2500 #?
img_size = 64
data_path = '/home/steven/CMPS/Images/pj_prepro/64x64'
X_train, Y_train_rgb, mean_image = util.load_databatch(data_path, 1, img_size)
num_input = X_train.shape[0]
for j in range(num_input):
        X_train[j] = util.preprocess_image(X_train[j])
L_train = X_train[:num_input-batch_size,:,:,0] #L-channel only
ab_train = X_train[:num_input-batch_size,:,:,1:] #ab-channels
L_test = X_train[num_input-batch_size:,:,:,0] #L-channel only
ab_test = X_train[num_input-batch_size:,:,:,1:] #ab-channels

#Build the estimator
model = tf.estimator.Estimator(model_fn)

#Define the input function for training 
input_fn = tf.estimator.inputs.numpy_input_fn(x={'images':L_train}, y=ab_train,
batch_size=batch_size, num_epochs=None, shuffle=True)

# Train the Model
model.train(input_fn, steps=num_steps)


#Evaluate Model
#Input test  
input_fn = tf.estimator.inputs.numpy_input_fn(x={'images':L_test}, y=ab_test,
batch_size=batch_size, num_epochs=None, shuffle=False)
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])